In [ ]:
import pandas as pd
import altair as alt
import numpy as np
from pathlib import Path
import re

In [ ]:
root_path = Path('/cw/dtaidata/ml/2023-irishElectricityProfiles/london/original_dataset/csv/data_collection/data_tables')
result_path = Path('/cw/dtaidata/ml/2023-irishElectricityProfiles/london/raw_data')
preprocessed_path = Path('/cw/dtaidata/ml/2023-irishElectricityProfiles/london/preprocessed')
result_path.mkdir(exist_ok = True)
preprocessed_path.mkdir(exist_ok = True)

## Data_df

In [ ]:
new_index = pd.date_range('2012-01-01 00:00', '2012-12-31 23:30', freq = '30min')
data_df = (
    # read all data files and concat
    pd.concat((pd.read_csv(path, index_col = 0) for path in root_path.glob('consumption*')), axis = 1)
    
    # make index datetime 
    .pipe(lambda x: x.set_axis(pd.to_datetime(x.index), axis = 0))

)

raw_data_df = (
    # start from the data df
    data_df
    
    # transpose 
    .T 
    
    # name axis 
    .sort_index()
    .rename_axis('meterID', axis = 0)
    .rename_axis('timestamp', axis = 1)
    
    # drop the incomplete days at the start and end
    .loc[:, '2011-11-24':'2014-02-27 23:30']
)


display(raw_data_df.head())
raw_data_df.to_pickle(result_path/'raw_data_df.pkl')


# Read info df and give columns names

In [ ]:
questions = pd.read_csv(root_path/'survey_questions.csv', index_col = 0).drop(columns = 'Survey')
questions.head()

In [ ]:
info_df = (
    # read the survey answers
    pd.read_csv(root_path/'survey_answers.csv', low_memory = False, index_col = 0)
    
    # select appliance questionaire
    .loc[:, 'Q211':]
    
    # give the columns a better name based on the question dataframe
    .stack()
    .to_frame('answer')
    .rename_axis(index = ['meterID', 'Question_id'])
    .join(questions.shift(-1))
    .reset_index()
    .assign(
        question = lambda x: x['Question_id'].str.cat(x['Question'], sep = ': ')
    )
    .set_index(['meterID', 'question'])
    .drop(columns = ['Question_id', 'Question'])
    .unstack()
    .droplevel(0, axis = 1)
    .iloc[:, 1:]
)

info_df.head()

# Preprocess questionair into features

In [ ]:
# store all preprocessed features here 
features = pd.DataFrame(index = info_df.index)

## Features to keep and just rename

In [ ]:
APPLIANCE_COLUMNS =['Q279: No. Fridges: type 1', 'Q280: No. Fridges: type 2',
       'Q281: No. Fridges: type 3', 'Q282: No. Fridges: type 4',
       'Q283: No. Freezers: type 1', 'Q284: No. Freezers: type 2',
       'Q285: No. Freezers: type 3', 'Q286: No. Freezers: type 4',
       'Q287: No. Freezers: type 5', 'Q288: No. Freezers: type 6',
       'Q289: No. Fridge-Freezers: type 1',
       'Q290: No. Fridge-Freezers: type 2',
       'Q291: No. Fridge-Freezers: type 3',
       'Q292: No. Fridge-Freezers: type 4', 'Q293: No. Electric hob',
       'Q294: No. Gas hob', 'Q295: No. Electric oven', 'Q296: No. Microwave',
       'Q297: No. Washing machine (not with dryer)', 'Q298: No. Tumble dryer',
       'Q299: No. Washer-dryer', 'Q300: No. Dishwasher',
       'Q301: No. Electric shower',
       'Q302: No. Over-sink electric water heater',
       'Q303: No. Portable electric heater', 'Q304: No. Television',
       'Q305: No. Desktop PC/computer', 'Q306: No. Laptop computer',
       'Q307: No. Printer', 'Q308: No. Router (for broadband internet)',
       'Q309: No. Dvd/blu-ray player', 'Q310: No. Cable TV box (e.g., Virgin)',
       'Q311: No. Satellite TV box (E.g., Sky)', 'Q312: No. Freeview TV box',
       'Q313: No. Games console', 'Q314: No. Standby savers']
appliance_rename = {key: key[10:] for key in APPLIANCE_COLUMNS}

TV_COLUMNS = [ 'Q315: TV Type 1', 'Q316: TV Type 2', 'Q317: TV Type 3',
       'Q318: TV Type 4', 'Q319: TV Type 5', 'Q320: TV Type 6']
tv_rename = {key: key[6:] for key in TV_COLUMNS} 

manual_rename =  {
    'Q213: Household Size': 'nb_of_inhabitants', 
    'Q234: Work from home': 'work_from_home', 
    'Q238: Rooms in home': 'nb_of_rooms',
    'Q239: Bedrooms': 'nb_of_bedrooms', 
    'Q240: Insulation: Double glazing': 'insulation_double_glazing', 
    'Q241: Insulation: Roof or loft insulation': 'insulation_roof', 
    'Q242: Insulation: Wall insulation': 'insulation_walls',
    'Q243: Insulation: Floor insulation': 'insulation_floor',
    'Q244: Insulation: Hot water tank insulation/lagging': 'insulation_warm_water_tank', 
}

appliance_rename = {**appliance_rename, **tv_rename, **manual_rename}

new_features = info_df[appliance_rename.keys()].rename(columns = appliance_rename)
print('features kept without preprocessing:')
with pd.option_context('display.max_rows', None, 'display.max_columns', 2) : 
    display(new_features.T)

features = features.join(new_features)

info_df = info_df.drop(columns = appliance_rename.keys())

## Number of males and females

In [ ]:
GENDER_COLUMNS = ['Q214: Household Member 1 Gender', 'Q215: Household Member 2  Gender',
       'Q216: Household Member 3 Gender', 'Q217: Household Member 4 Gender',
       'Q218: Household Member 5 Gender', 'Q219: Household Member 6 Gender',
       'Q220: Household Member 7 Gender', 'Q221: Household Member 8 Gender']


gender_info = info_df[GENDER_COLUMNS]
print('original')
display(gender_info.head())

features['nb_of_males'] = (gender_info == 'Male').sum(axis = 1)
features['nb_of_females'] = (gender_info == 'Female').sum(axis = 1)

print('derived features')
display(features[['nb_of_males', 'nb_of_females']].head())

info_df = info_df.drop(columns = GENDER_COLUMNS)

## Min, max and mean inhabitant age

In [ ]:
AGE_COLUMNS = ['Q222: Household Member 1 Age', 'Q223: Household Member 2  Age',
       'Q224: Household Member 3 Age', 'Q225: Household Member 4 Age',
       'Q226: Household Member 5 Age', 'Q227: Household Member 6 Age',
       'Q228: Household Member 7 Age', 'Q229: Household Member 8 Age']

age = (
    info_df.loc[:, AGE_COLUMNS]
)

print('original')
display(age.head())

ages = age.stack().unique()
ages.sort()
replace_dict = {
    "Don't know": np.NAN
}
for key in ages[:-1]: 
    result = re.findall(r'\d+', key)
    replace_dict[key] = np.array(result).astype(float).mean()
replace_dict

age_features = (
    age.replace(replace_dict)
    .agg(['mean', 'min', 'max'], axis = 1)
    .set_axis(['mean_age', 'min_age', 'max_age'], axis = 1)
    # fill missing with mean of column
    .pipe(lambda df: df.fillna(df.mean(axis = 0)))
)

display('derived features')
display(age_features)

# add the the features dataframe
features = features.join(age_features)

info_df = info_df.drop(columns = AGE_COLUMNS)

## Home ownership feature

In [ ]:
home_ownership_replace_dict = {
    'Rents (with or without housing benefit) - from private landlord' : 'Rents' ,
    'Owns outright' : 'Owns' ,
    'Rents (with or without housing benefit) - from local authority/Council or Housing Association' : 'Rents' ,
    'Part owns and part rents (shared ownership)' : 'Shared ownership' ,
    'Owns with a mortgage or loan' : 'Owns' ,
    'Other' : 'Other' ,
    'Lives here rent-free' : 'Rent-free' ,
    'Other Other: LEASEHOLDER' : 'Rents' ,
    'Part owns and part rents (shared ownership) Other: HOUSING ASSOCIATION' : 'Rents' ,
    'Other Other: comes with job' : 'Rents' ,
    'Rents (with or without housing benefit) - from private landlord Other: SHELTERED ACCOMMODATION' : 'Rents' ,
    'Rents (with or without housing benefit) - from local authority/Council or Housing Association Other: SHARED OWNERSHIP' : 'Shared ownership' ,
    'Other Other: LEASE HOLDER' : 'Rents' ,
    'Other: RESIDENTIAL HOME' : 'Rents' ,
    'Other Other: COUNCIL' : 'Rents' ,
    'Other Other: RENTS FROM HM FORCES' : 'Rents' ,
    'Other Other: LEASEHOLD/OWNERSHIP' : 'Rents' ,
    'Other: LEASEHOLD' : 'Rents' ,
    'Other Other: SHELTERED HOUSING' : 'Rents' ,
    'Other Other: Rather not say' : 'Missing' ,
    'Owns outright Other: SHELTERED ACCOMMODATION' : 'Rents' ,
    'Other Other: RENT FROM LOCAL HOUSING CO-OPERATIVE' : 'Rents' ,
    'Part owns and part rents (shared ownership) Other: SHELTERED HOUSING' : 'Rents' ,
}

print('before preprocessing')
display(info_df['Q231: Newspapers - Printed'].value_counts(dropna=False).to_frame('count'))


# weird error in the column naming 
features['home_ownership'] = info_df['Q231: Newspapers - Printed'].replace(home_ownership_replace_dict)

print('after preprocessing') 
display(features['home_ownership'].value_counts(dropna = False).to_frame('count'))

info_df = info_df.drop(columns = 'Q231: Newspapers - Printed')

## Building type

In [ ]:
house_replacement = {
    'Terraced - middle': 'house-terraced', 
    'Terraced - end': 'house-semi-detached', 
    'Semi-detached': 'house-semi-detached', 
    'Detached': 'house-detached'
}

flat_replacement = {
    'In a purpose built block or tenement': 'flat',
	'Part of a converted house/building': 'flat',
	'In a commercial building, (for example in an office building, hotel, or over a shop)': 'flat',
}

mobile_replacement = {
    'A caravan or other mobile or temporary structure': 'mobile',
}

BUILDING_COLUMNS = ['Q235: Accomodation: House', 'Q236: Accomodation: Flat', 'Q237: Accomodation: Mobile']

print('before preprocessing')
display(info_df[BUILDING_COLUMNS])

building_type = (
    info_df[BUILDING_COLUMNS]
    .set_axis(['house', 'flat', 'mobile'], axis = 1)
    .replace(house_replacement)
    .replace(flat_replacement)
    .replace(mobile_replacement)
)
# fill in the correct values (some people filled some things twice)
building_type.loc[lambda x: ~x.flat.isna(), 'house'] = building_type.loc[lambda x: ~x.flat.isna(), 'flat']
building_type.loc[lambda x: ~x.mobile.isna(), 'house'] = building_type.loc[lambda x: ~x.mobile.isna(), 'mobile']

building_type_feature = building_type.house.to_frame('building_type')
building_type_feature

print('after preprocessing')
display(building_type_feature) 

features = features.join(building_type_feature)

info_df = info_df.drop(columns = BUILDING_COLUMNS)

## Central heating type

In [ ]:
raw_heating = info_df['Q246: Central heating']

print('before_preprocessing')
display(raw_heating.value_counts(dropna = False).to_frame('count'))

central_heating_features = (
    pd.DataFrame(index = info_df.index, columns = ['gas', 'electric', 'other', 'solid', 'oil'])
    .assign(
        gas = raw_heating.str.contains('Gas'), 
        electric = raw_heating.str.contains('Electric'), 
        other = raw_heating.str.contains('Other central heating'), 
        solid = raw_heating.str.contains('Solid fuel'), 
        oil = raw_heating.str.contains('Oil')
    )
    .pipe(lambda x: x.set_axis('heating_'+x.columns, axis = 1))
)
print('after preprocessing')
display(central_heating_features)

features = features.join(central_heating_features)

info_df = info_df.drop(columns = 'Q246: Central heating')

## Heating control features

In [ ]:
# if string contains key, replace with value
replacement_dict = {
    'switches on and off automatically at set times': 'heating_control_automatic_timed', 
    'manually at the boiler when needed': 'heating_control_manual_boiler',
    'automatically by a thermostatic': 'heating_control_automatic_temperature', 
    'control room temperature using thermostatic valves on the radiators': 'heating_control_manual_valves', 
    "I'm not sure": 'heating_control_unknown',
    "nan": 'heating_control_unknown'
}


raw_heating_control = info_df.loc[:, 'Q247: Central heating - control'].astype('str')

print('before preprocessing') 
display(raw_heating_control.value_counts(dropna = False).to_frame('count')) 

heating_control_features = pd.DataFrame(index = info_df.index)

for key,value in replacement_dict.items(): 
    if value not in heating_control_features.columns:
        heating_control_features[value] = raw_heating_control.str.contains(key)
    else: 
        heating_control_features[value] = heating_control_features[value] | raw_heating_control.str.contains(key)
heating_control_features

features = features.join(heating_control_features)
print('after preprocessing')
display(heating_control_features) 

info_df = info_df.drop(columns ='Q247: Central heating - control')

## Water heating

In [ ]:
# if string contains key, fill in value
replacement_dict = {
    'Hot water storage tank with gas boiler - used for both central heating and hot water': 'gas_w_storage',
    "Gas boiler (without hot water storage tank) - used for both central heating and hot water ('combi' boiler)": 'gas', 
    "Hot water storage tank with electric immersion heater": 'electric_w_storage', 
    "Hot water storage tank with gas boiler - used for hot water only": 'gas_w_storage',
    "Gas boiler (without hot water storage tank) - used for hot water only": 'gas', 
    "Other": 'other', 
    "Don't know": np.nan, 
    np.nan: np.nan, 
}

print('before preprocessing') 
display(info_df['Q248: Heating water'].value_counts(dropna = False).to_frame('value'))

heating_water_features = (
    info_df['Q248: Heating water']
    .astype('str')
    # some literal matching 
    .mask(lambda x: x.str.contains('Hot water storage tank with gas boiler - used for both central heating and hot water'), 'gas_w_storage')
    .mask(lambda x: x.str.contains("Gas boiler (without hot water storage tank) - used for both central heating and hot water", regex = False), 'gas')
    .mask(lambda x: x.str.contains("Hot water storage tank with electric immersion heater"), 'electric_w_storage')
    .mask(lambda x: x.str.contains("Hot water storage tank with gas boiler - used for hot water only", regex = False), 'gas_w_storage')
    .mask(lambda x: x.str.contains("Gas boiler (without hot water storage tank) - used for hot water only", regex = False), 'gas')
    # if other but gas mentioned -> gas 
    .mask(lambda x: x.str.contains("Other") & x.str.lower().str.contains('gas'), 'gas')
    # if other but electric mentioned -> electric 
    .mask(lambda x: x.str.contains("Other") & x.str.lower().str.contains('electric'), 'electric_w_storage')
    # remaining others are NaN
    .mask(lambda x: x.str.contains("Don't know") | x.str.contains('Other') , np.nan)
    .to_frame('water_heating')
)
print('after_preprocessing') 
display(heating_water_features.value_counts().to_frame('count'))

features = features.join(heating_water_features) 
info_df = info_df.drop(columns = 'Q248: Heating water')

## Irrelevant columns
Everything that remains is 'irrelevant'

In [ ]:

with pd.option_context('display.max_rows', None, 'display.max_columns', 2) : 
    display(info_df.T)

## preprocessed features

Overview of the preprocessed features

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 2) : 
    display(features.T)
info_df = features

In [ ]:
info_df.to_pickle(result_path/'raw_info_df_features.pkl')

# Final preprocessing for the tree models

## Gender columns are OK

## Water heating, fill NaNs as value between electrical and gas 

In [ ]:
print('before preprocessing') 
display(info_df.water_heating.value_counts(dropna = False))

info_df['water_heating'] = info_df['water_heating'].replace({
    'electric_w_storage': 1, 
    np.NaN: 1.5, 
    'nan': 1.5, 
    'gas':2, 
    'gas_w_storage': 3
})

print('after preprocessing')
info_df.water_heating.value_counts(dropna = False)

## Fill NaN with means for nb_of_inhabitants, nb_of_rooms, nb_of_bedrooms

In [ ]:
print('before preprocessing') 
display(info_df.nb_of_inhabitants.value_counts(dropna = False))

columns = ['nb_of_inhabitants', 'nb_of_rooms', 'nb_of_bedrooms']
info_df = info_df.fillna(info_df[columns].mean(axis = 0))
print('after preprocessing')
info_df.nb_of_inhabitants.value_counts(dropna = False)

## Home ownership: fill NaN's and encode as one_hot_encoding

In [ ]:
print('before preprocessing') 
info_df.home_ownership.value_counts(dropna = False)

In [ ]:
info_df['home_ownership'] = info_df['home_ownership'].replace({np.NaN: 'Missing'}).str.lower().str.replace(' ', '_')
info_df.home_ownership.value_counts(dropna = False)

home_ownership_features = pd.get_dummies(info_df['home_ownership'], prefix = 'home_ownership')

info_df = (
    info_df
    .drop(columns = 'home_ownership')
    .join(home_ownership_features)

)

print('after preprocessing') 
home_ownership_features.head()

## Work_from_home fill NaN's with -1 and encode as ordinal

In [ ]:
print('before preprocessing')
info_df.work_from_home.value_counts(dropna = False)

In [ ]:
info_df['work_from_home'] = info_df['work_from_home'].replace({
    np.NaN: -1, 
    'Never': 0, 
    'Occasionally': 1, 
    'About half the time': 2, 
    'Most/all weekdays': 3
})

In [ ]:
print('after preprocessing')
info_df.work_from_home.value_counts(dropna = False)

## Building type to ordinal

In [ ]:
print('before preprocessing') 
info_df.building_type.value_counts(dropna = False)

In [ ]:
info_df['building_type'] = info_df['building_type'].replace({
    np.NaN: -1, 
    "mobile": 0, 
    "flat":1, 
    'house-terraced': 2, 
    'house-semi-detached':3, 
    'house-detached': 4
})

In [ ]:
print('after preprocessing') 
info_df.building_type.value_counts(dropna = False)

## TVs to count of TVs per type 

In [ ]:
tv_type_columns = ['TV Type 1', 'TV Type 2', 'TV Type 3', 'TV Type 4',
       'TV Type 5', 'TV Type 6']

print('before preprocessing') 
info_df[tv_type_columns].head()

In [ ]:
tv_counts = (
    info_df[tv_type_columns]
    .replace({"Don't know":'Unknown', 'Traditional/older style (CRT)':'crt'})
    .apply(lambda x: x.value_counts(),axis = 1)
    # fill NaN's with 0 
    .fillna(0)
    .rename(columns = lambda x: f'tv_count_{x.lower()}')
)


info_df = (
    info_df
    .drop(columns = tv_type_columns)
    .join(tv_counts)
)

print('after preprocessing') 
tv_counts.head()

## Insulation to 0, 0.5, 1 (False, Missing, True)

In [ ]:
columns = ['insulation_double_glazing', 'insulation_roof', 'insulation_walls', 'insulation_floor', 'insulation_warm_water_tank']

In [ ]:
info_df[columns] = info_df[columns].replace({
    'Yes':1, 
    'No': 0, 
    np.NaN: 0.5, 
    "Don't know": 0.5, 
})

## Heating columns to 0, 0.5, 1 (False, Missing, True)

In [ ]:
heating_columns = ['heating_gas',
       'heating_electric', 'heating_other', 'heating_solid', 'heating_oil',
       'heating_control_automatic_timed', 'heating_control_manual_boiler',
       'heating_control_automatic_temperature',
       'heating_control_manual_valves', 'heating_control_unknown']
info_df[heating_columns].stack(dropna = False).value_counts(dropna = False)

In [ ]:
info_df[heating_columns] = info_df[heating_columns].replace({
    np.NaN: 0.5, 
    False: 0, 
    True: 1
})

## The fridge freezer and appliance columns are ok

In [ ]:
fridge_columns = ['Fridges: type 1', 'Fridges: type 2', 'Fridges: type 3',
       'Fridges: type 4', 'Freezers: type 1', 'Freezers: type 2',
       'Freezers: type 3', 'Freezers: type 4', 'Freezers: type 5',
       'Freezers: type 6', 'Fridge-Freezers: type 1',
       'Fridge-Freezers: type 2', 'Fridge-Freezers: type 3',
       'Fridge-Freezers: type 4']
info_df[fridge_columns].stack().value_counts(dropna = False)

In [ ]:
appliance_columns = ['Electric hob', 'Gas hob', 'Electric oven', 'Microwave',
       'Washing machine (not with dryer)', 'Tumble dryer', 'Washer-dryer',
       'Dishwasher', 'Electric shower', 'Over-sink electric water heater',
       'Portable electric heater', 'Television', 'Desktop PC/computer',
       'Laptop computer', 'Printer', 'Router (for broadband internet)',
       'Dvd/blu-ray player', 'Cable TV box (e.g., Virgin)',
       'Satellite TV box (E.g., Sky)', 'Freeview TV box', 'Games console',
       'Standby savers']
info_df[appliance_columns].stack().value_counts(dropna = False)

## Final state

In [ ]:
info_df.columns = info_df.columns.str.lower().str.replace(': ', '_').str.replace(' ', '_')
info_df = info_df.rename(columns = {'cable_tv_box_(e.g.,_virgin)': 'cable_tv_box', 'satellite_tv_box_(e.g.,_sky)': 'satellite_tv_box'})
with pd.option_context('display.max_rows', None,
                       'display.max_columns', 3):
    display(info_df.T)
info_df.to_pickle(result_path/'raw_info_df_numerical.pkl')

# Preprocess to final version

In [ ]:
from util import transform_raw_data_and_save, check_dataset

yearly_data_df, daily_data_df, yearly_info_df, daily_info_df = transform_raw_data_and_save(
    raw_data_df = result_path/'raw_data_df.pkl', 
    yearly_info_df = result_path/'raw_info_df_numerical.pkl', 
    result_path = preprocessed_path, 
    weather_city = 'London',
    holiday_country = 'England', 
    year_to_use_as_index = 2012
)
check_dataset((yearly_data_df, daily_data_df, yearly_info_df, daily_info_df))

In [ ]:
yearly_data_df

In [ ]:
daily_data_df

In [ ]:
yearly_info_df

In [ ]:
daily_data_df